# 7PAVITPR: Introduction to Statistical Programming
# Python practical 12


_Department of Biostatistics and Health Informatics<br/>
Institute of Psychiatry, Psychology and Neuroscience<br/>
King's College London<br/>_


_Acknowledgment: based on parts of the course "Big Data Analytics in Python", written by Sagar Jilka, KCL_

# Further data handling packages: plotting and statistics

This practical is a demonstration of plotting and statistics from the follwoing packages:

* MatPLotlib
* seaborn
* SciPy
* statsmodels

There is no coding for you to do - it is a demo for you to read and run, giving examples of how to use these packages.

__NB You need to run every cell!__

# Preparing the data

We will use the same brain size and IQ data sets as from practical 11. We will repeat the main code from that practical to read in the data, merge it and deal with missing values.

In [ ]:
# Import pandas...
import pandas as pd

# Read in the data
df_m = pd.read_csv('brain_clinic_measures.csv')
df_p = pd.read_csv('brain_participant_info.csv')

# Drop unwanted columns
df_m = df_m.drop('Unnamed: 0', axis=1)
df_p = df_p.drop('Unnamed: 0', axis=1)

# create one merged DataFrame
df_all = df_m.merge(df_p, on = ['ppt_id'], how = 'left')

# Transform dataframe to means for gender
means_for_gender = df_all.groupby("Gender").transform("mean", numeric_only=True)

# Fill in Height and Weight NAs with mean for gender
df_all = df_all.fillna(means_for_gender.loc[:,['Height','Weight']])

# Take a look
# df_all.head()
df_all.describe()

# Plotting and visualising data

Start with basic bar/line plots, learning seaborn methods, and then something interactive

In [ ]:
#we need to import more libraries, notably matplotlib and seaborn

import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns

#note the magic function below. This is important because it will allow the plots you
#create to appear here in the notebook.
%matplotlib inline


In [ ]:
xx = df_all["FSIQ"] # you can replace FSIQ with any other column.. try it
plt.hist(xx, bins = 10);

Now lets consider the seaborn library, which we will use as seaborn is generally easier, and make the images much better!

In [ ]:
sns.distplot(df_all['Weight'], kde=False, rug=True, bins = 10)

We want to investigate:


(1) Differences in IQ/brain size between men and women?

(2) Any correlations?

** Now is a good time to explore the seaborn library.
I will show you how to use the examples and apply them to your work **

In [ ]:
#https://seaborn.pydata.org/generated/seaborn.catplot.html?highlight=catplot#seaborn.catplot

sns.catplot(data=df_all,
            x="Gender",
            y="PIQ",
            #palette={"Male": "blue", "Female": "pink"}, # you can also specify colours!
            kind="strip"); #experiment with different kinds, e.g. “bar”, “strip”, “swarm”, “box”, “violin”, or “boxen”.



In [ ]:
# Lets add another categorical variable to the graph:
# We want to add Diagnosed or not to the figure

sns.catplot(data=df_all,
            x="Gender",
            y="PIQ",
            hue = "Diagnosed",
            #palette={"Male": "blue", "Female": "pink"}, # you can also specify colours!
            kind="bar"); #experiment with different kinds, e.g. “bar”, “strip”, “swarm”, “box”, “violin”, or “boxen”.



In [ ]:
# Lets do some correlations and make a correlation matrix

corr = df_all.corr(numeric_only=True)

In [ ]:
corr

In [ ]:
# Now lets add some colour to help differentiate between variables

corr.style.background_gradient().set_precision(2)

If you want to calculate pearson r then you can use the function below, don't worry about the code

In [ ]:
from scipy.stats import pearsonr
import pandas as pd

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues

In [ ]:
calculate_pvalues(df_all)

In [ ]:
#Exploring scatterplots

#https://seaborn.pydata.org/generated/seaborn.scatterplot.html

In [ ]:
sns.scatterplot(x="FSIQ", y="PIQ", data=df_all)

In [ ]:
#plt.figure(figsize=(10,8))

sns.scatterplot(data=df_all,
                x="FSIQ",
                y="PIQ",
                hue="Gender",) #experiment with different categorical 'hues', e.g. Diagnosed
                #size = "Height") # Explore what the size argument does...


In [ ]:
# Saving your figures for publication

# assign the code to a random variable (something like g = )

g = sns.scatterplot(data=df_all,
                x="FSIQ",
                y="PIQ",
                hue="Gender",) #experiment with different categorical 'hues', e.g. Diagnosed
                #size = "Height") # Explore what the size argument does...


# Use the savefig argument and provide a filename, such as figure1.png
# You can use keyword arguments based on your journal's submission requirements
# For exmaple, for the British Journal of Psychiatry (https://www.cambridge.org/core/services/authors/journals/journals-artwork-guide)
# they want images at 300dpi and ideally in TIFF format, so you can save your figure appropriately
# So check with your submission guidelines and provide the appropriate arguments
# Tip: for posters, you might want to use the transparancy argument

g.figure.savefig("output.tiff", dpi = 300, transparent=True)

In [ ]:
df_all.head()

# Some statistics in Python

##### IV = Gender (Male, Female), also Diagnoses (Yes, No)
##### DV = IQ Scores (FSIQ, VIQ, PIQ)

We want to test if there is a difference in IQ measures between genders? Or "diagnosed"? What tests can we use?

In [ ]:
# There are two libraries (that i know of anyway) to do stats in Python - scipy and statsmodels

# Lets start with scipy and we will compare the syntax and output with statsmodel

# Import as below...

import scipy.stats as stats

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns

### Assumption of normality

#### Shapiro-wilk test (output = w test statistic, p value)

In [ ]:
print(stats.shapiro(df_all["FSIQ"]))

# consider the difference between the above and the below commented code:

print(stats.shapiro(df_all["FSIQ"][df_all['Gender'] == 'Male']))

In [ ]:
# You can also test other DVs

print(stats.shapiro(df_all["PIQ"]))
print(stats.shapiro(df_all["VIQ"]))

In [ ]:
# Now lets make some Q-Q Plots
stats.probplot(df_all["FSIQ"],plot= plt)

# Give your figure a title
plt.title("FSIQ Q-Q Plot");

In [ ]:
# Lets make a loop and test all three of our IQ DVs in one go:

# First make a list variable called cols with your DVs in it
cols = ['FSIQ', 'PIQ', 'VIQ']

# Then make a loop using the for statement. This will loop through every item in cols
# and do you tell it to do, in this case, we are telling it to do stats.shapiro
for i in cols:
    print(i)
    print(stats.shapiro(df_all[i][df_all['Gender'] == 'Male']))

for i in cols:
    stats.probplot(df_all[i][df_all['Gender'] == 'Male'], plot= plt)
    plt.title("Mental Health Q-Q Plot")

print("\n\nAssumption of normality is violated as (all) the p-values are < than 0.05.")

##### Levene's Test

In [ ]:
levene_1 = df_all["PIQ"][df_all['Gender'] == 'Male']
levene_2 = df_all["PIQ"][df_all['Gender'] == 'Female']

In [ ]:
stats.levene(levene_1, levene_2)

In [ ]:
# Lets make a quick loop so we can run a levene's test on all our DVs

for i in cols:
    print(i , ':' , stats.levene(df_all[i][df_all['Gender'] == 'Male'],
                                 df_all[i][df_all['Gender'] == 'Female']))

##### ANOVA

In [ ]:
# We will statsmodels because the output is better (more readable)

import statsmodels.api as sm
from statsmodels.formula.api import ols

# Documentation here: https://www.statsmodels.org/stable/index.html

In [ ]:
results = ols("FSIQ ~ C(Gender)", data = df_all).fit()

In [ ]:
results.summary()

In [ ]:
print("The adfsaf asdf , F(%f, %f) = %f , p = %f" %(results.df_resid,
                                                    results.df_model,
                                                    results.fvalue,
                                                    results.f_pvalue))

In [ ]:
# Consider writing a function that writes out your results section for you!
# You need to find the following bits of info from the above...

# F(df effect, df error) = F-value, MSE = mean-square error, p-value".
# e.g., "IQ scores did/didn't differ significantly between genders, F(X,XX) = XXXX, MSE = XXX, p = XXX.

# You can find each of those individual bits of data by typing results. then hit tab and
# you will be able to see all the methods that the results object contains!

# For instance:

#print(results.df_model)
#print(results.df_resid)
#print(results.fvalue)
#print(results.f_pvalue)
#print(results.mse_total)

# Now you just need to put all that together...

In [ ]:
def res_output(dv, iv, df):

        # So first make two variables that represent the IV and DV
        x = ("~ C(%s)" %iv)
        y = str(dv + x)

        # Then make the model.
        results = ols(y, data=df).fit()

        # Then make a statement which prints out an appropriate statement
        # based on the p-value...

        if results.f_pvalue > 0.05:

            print("A one-way ANOVA was conducted to compare difference in %s between %s. We found no significant difference between %s,\
                   F(%f, %f) = %f , p = %f" %(dv, iv, iv,
                                              results.df_model,
                                              results.df_resid,
                                              results.fvalue,
                                              results.f_pvalue))
        else:
            print("A one-way ANOVA was conducted to compare difference in %s between %s. We found a significant difference between %s\
                   F(%f, %f) = %f , p = %f" %(dv, iv, iv,
                                              results.df_model,
                                              results.df_resid,
                                              results.fvalue,
                                              results.f_pvalue))

        # If you fancy, you can tell the function to return an output such as the
        # summary table, if so, uncomment the bottom bit below...

        return results.summary()

In [ ]:
# Now you can call your function and give it your arguments like below...

res_output(dv = "VIQ",
           iv = "Gender",
           df = df_all)